# E-commerce Web Scraping

## Business Problem 📝💹

En este proyecto trabajamos para una compañía (Whirpool) de e-commerce que se prepara para un fin de semana de promociones. Como tarea nos fue asignado averigüar los precios que maneja la competencia de algunos productos especiales. Nuestro objetico es **determinar el precio mínimo** que maneja la competencia para cada producto para así poder igualarlo. 

Stakeholders: 
* Marketing manager

Recibimos:
* Un archivo de excel (Products_and_comp.xlsx) con una lista de los productos sobre los cuales debemos hacer web scraping.
* Los nombres de las compañías que son competencia y sobre las cuales debemos ingresar a su portal de e-commerce.

Entregable:
* Un pandas dataFrame con los precios y los URL de los productos para cada compañía de la competencia.

In [1]:

import pandas as pd
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re

# Creamos un entorno virtual y usamos las siguientes librerias fueron usadas:
# from selenium import webdriver 
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_excel('Products_and_comp.xlsx')
df.shape

(424, 35)

In [3]:
df

Material       Marca         Sub-Categoría  Liverpool  Liverpool URL  \
0      WC11016Q   WHIRLPOOL          Congeladores        NaN            NaN   
1    KSM150PSER  KITCHENAID             Batidoras        NaN            NaN   
2     KP26M1XOB  KITCHENAID             Batidoras        NaN            NaN   
3       WK5053Q   WHIRLPOOL         Despachadores        NaN            NaN   
4       WM1807B   WHIRLPOOL  Hornos de Microondas        NaN            NaN   
..          ...         ...                   ...        ...            ...   
419     AT1130F       ACROS                   NaN        NaN            NaN   
420     AT1330D       ACROS           Dos puertas        NaN            NaN   
421     WA4226Q   WHIRLPOOL            Frío/Calor        NaN            NaN   
422     AM1807B       ACROS                   NaN        NaN            NaN   
423    WWR7000S   WHIRLPOOL                   NaN        NaN            NaN   

     Coppel  Coppel URL  Home Depot  Home Depot URL  Costco  ...  REAMI  \
0       NaN         NaN         NaN             NaN     NaN  ...    NaN   
1       NaN         NaN         NaN             NaN     NaN  ...    NaN   
2       NaN         NaN         NaN             NaN     NaN  ...    NaN   
3       NaN         NaN         NaN             NaN     NaN  ...    NaN   
4       NaN         NaN         NaN             NaN     NaN  ...    NaN   
..      ...         ...         ...             ...     ...  ...    ...   
419     NaN         NaN         NaN             NaN     NaN  ...    NaN   
420     NaN         NaN         NaN             NaN     NaN  ...    NaN   
421     NaN         NaN         NaN             NaN     NaN  ...    NaN   
422     NaN         NaN         NaN             NaN     NaN  ...    NaN   
423     NaN         NaN         NaN             NaN     NaN  ...    NaN   

     REAMI URL  Soriana  Soriana URL  La Unica  La Unica URL  Cimaco  \
0          NaN      NaN          NaN       NaN           NaN     NaN   
1          NaN      NaN          NaN       NaN           NaN     NaN   
2          NaN      NaN          NaN       NaN           NaN     NaN   
3          NaN      NaN          NaN       NaN           NaN     NaN   
4          NaN      NaN          NaN       NaN           NaN     NaN   
..         ...      ...          ...       ...           ...     ...   
419        NaN      NaN          NaN       NaN           NaN     NaN   
420        NaN      NaN          NaN       NaN           NaN     NaN   
421        NaN      NaN          NaN       NaN           NaN     NaN   
422        NaN      NaN          NaN       NaN           NaN     NaN   
423        NaN      NaN          NaN       NaN           NaN     NaN   

     Cimaco URL  Cyberpuerta  Cyberpuerta Url  
0           NaN          NaN              NaN  
1           NaN          NaN              NaN  
2           NaN          NaN              NaN  
3           NaN          NaN              NaN  
4           NaN          NaN              NaN  
..          ...          ...              ...  
419         NaN          NaN              NaN  
420         NaN          NaN              NaN  
421         NaN          NaN              NaN  
422         NaN          NaN              NaN  
423         NaN          NaN              NaN  

[424 rows x 35 columns]

In [4]:
# Identificadores de los productos
productos = df['Material']

In [5]:
# Obtenemos la competencia
competencia = df.columns[3::2]
competencia

Index(['Liverpool', 'Coppel', 'Home Depot', 'Costco', 'Sears', 'Elektra',
       'Walmart', 'Sam's', 'Suburbia', 'Palacio', 'Famsa', 'REAMI', 'Soriana',
       'La Unica', 'Cimaco', 'Cyberpuerta'],
      dtype='object')

## Mercado Libre Mex

Empezamos haciendole web scraping a Mercado Libre. Usamos Selenium con un driver de Chrome 108. 
Es necesario ejecutar el script _Mercado_Libre.py_
URL = 'https://www.mercadolibre.com.mx/a/store/seagate'

### Test
Iniciamos un test con el primer producto.


In [8]:
resultados_test1 = pd.read_csv('test_ML.csv', index_col = [0])
resultados_test1

titles  prices  \
0  Canastilla Cesta Reja Organizdor Congelador Wh...     449   
1  Congelador horizontal Whirlpool WC11016Q blanc...  13,990   
2  Congelador Horizontal Whirlpool Wc11016q Blanc...   2,499   
3       Congelador 11 Pies Whirlpool Wc11016q Blanco  13,990   
4  Congelador Xpert Energy Saver 300.00 L / 11 P³...  16,777   
5        Congelador Whirlpool 16 Pies Cúbicos Blanco  12,000   

                                               links        id  
0  https://articulo.mercadolibre.com.mx/MLM-15094...  WC11016Q  
1  https://www.mercadolibre.com.mx/congelador-hor...  WC11016Q  
2  https://articulo.mercadolibre.com.mx/MLM-16796...  WC11016Q  
3  https://articulo.mercadolibre.com.mx/MLM-66872...  WC11016Q  
4  https://articulo.mercadolibre.com.mx/MLM-55919...  WC11016Q  
5  https://articulo.mercadolibre.com.mx/MLM-15711...  WC11016Q